# Delay processing

In this notebook, we will process the timetable data containing scheduled and actual data of transport throught Switzerland. We will use this data to compute average delays for each hour of the day, depending on the line and type of transport. 

## Start pyspark 

In [ ]:
%%local
import os
username = os.environ['JUPYTERHUB_USER']

get_ipython().run_cell_magic('configure', 
                             line="-f", 
                             cell='{ "name":"%s-project", "executorMemory":"4G", "executorCores":4, "numExecutors":10, "driverMemory": "4G" }' % username)

In [ ]:
%%send_to_spark -i username -t str -n username

In [ ]:
import pandas as pd
import pyspark
from pyspark.sql.functions import *

In [ ]:
# Read the data from hdfs
df = spark.read.orc("/data/sbb/orc/istdaten")

In [ ]:
folder_name = "hdfs:///user/theAggregators"

In [ ]:
# Read the processed vertices to get the stop ids
df_nodes = spark.read.csv("{}/vertices.csv".format(folder_name), header=True).select(col("stop_id"))

In [ ]:
# Keep node base prefix and drop platform information 
nodes = [n.stop_id.split(":")[0] for n in df_nodes.collect()]

## Computing the delays for each row

We have to only take into consideration delay that are positives: if the transport arrives in advance, we set the delay to zero.

In [ ]:
# UDF that clips delay to zero if it is negative
clip_neg = udf(lambda delay: 0 if delay < 0 else delay)

Next, we create a filtered dataframe in order to find all the transports where: 
- The actual and scheduled arrival time are given.
- The trip is not an additional trip and it did not fail.
- The stop is in the list of stops that we should consider (inside 15-km radius).

In [ ]:
# Filter the dataframe according to the criteria above
df_filtered = df.filter((df.ankunftszeit != "") & \
                       (df.an_prognose != "") & \
                       (df.zusatzfahrt_tf == "false") & \
                       (df.faellt_aus_tf == "false") & \
                       (df.bpuic.isin(nodes)))

From the filtered table, we only need:
- The name of the stop
- The operation id (betreiber_id) and the line id (linen_id) to identify a trip
- The id of the stop in the trip (bpuic) 
- The hour of the day during which the trip occurred, because we need a different lambda for each hour since the trafic can differ during the days

We also compute the delay by calculating the difference between the actual arrival time (`an_prognose`) and the scheduled arrival time (`ankunftszeit`).

Note that all delays are in seconds. The operator, line, stop id and the hour can help us compute the delay for each stop for a specific line. 

In [ ]:
# Select fields of interest and compute delay in seconds. Keep hours between 6 am and 9 pm.   
df_delay = df_filtered.select(
    df_filtered.haltestellen_name,
    df_filtered.betreiber_id, 
    df_filtered.linien_id, 
    df_filtered.bpuic,
    hour(to_timestamp(col("ankunftszeit"), "dd.MM.yyyy HH:mm")).alias("hour"),
    clip_neg((to_timestamp(col("an_prognose"), "dd.MM.yyyy HH:mm:ss").cast("long") \
                                - to_timestamp(col("ankunftszeit"), "dd.MM.yyyy HH:mm").cast("long"))) \
                                 .alias("delay")
    )\
    .filter((col("hour") >= 6) & (col("hour") < 21))

In [ ]:
df_delay.show(5)

## Computing the mean delay per hour 

Now that the dataframe is ready, we can groupy by the stop name, the operator, stop id and the hour to find the average delay for a specific stop from a line at a given time during the day. 

In [ ]:
# Group by stop name, line operator, stop id and hour
# Compute mean delay for each group
df_stop_line_delay = df_delay.groupBy(df_filtered.bpuic, df_delay.hour)\
                                .agg({"delay" : "mean"}).withColumnRenamed("avg(delay)", "delay")\
                                .select(
                                    col("bpuic"),
                                    col("hour"),
                                    col("delay")
                                )\
                                .cache()

In [ ]:
df_stop_line_delay.show(5)

Let's check how many of the original stops our delay information covers:

In [ ]:
# Number of tuples we have delay info for 
df_stop_line_delay.count()

In [ ]:
# Number of nodes our delay information covers
df_stop_line_delay.select(col("bpuic")).distinct().filter(col("bpuic").isin(nodes)).count()

In [ ]:
# Number of nodes we are considering
len(nodes)

We have delay parameters for 22970 tuples (bpuic, hour, delay) covering 1483 stops out of the 2306 that we are considering.

## Adding delay information to the list of edges

Now that we computed the average delay per stop, line, and hour, we will add this information to the edges of the graph we previously constructed. 

In [ ]:
# Convert the previous result into a pandas df
delays = df_stop_line_delay.toPandas()

In [ ]:
# Read the edges 
edges = spark.read.csv("{}/edges.csv".format(folder_name), header=True).toPandas()

In [ ]:
# Convert some fields to their correct type, to avoid mistakes later on
delays = delays.astype({'bpuic' : str, 'hour': int})
edges = edges.astype({'dst' : str, 'src': str, 'agency_id': str, 'route_short_name': str})

In [ ]:
# Create a column corresponding to the hour of the day
edges['hour'] = edges.dst_arrival.apply(lambda x: int(x.split(':')[0]))
# Parse the destination stop ids to ignore the platform number
edges["simple_dst_id"] = edges.dst.apply(lambda x: x.split(":")[0])

# Truncate the bpuic to have the same format as in edges
delays["len_bpuic"] = delays.bpuic.apply(lambda x: len(x))
delays.loc[delays.len_bpuic == 9, "bpuic"] = delays.loc[delays.len_bpuic == 9, "bpuic"].apply(lambda x : x[:-2])
delays = delays.drop(columns="len_bpuic")

We combine the two dataframes, to associate the destination of every edge to a delay: i.e the average delay that the train will have when arriving at the end of the edge. 

In [ ]:
# Combine the two dataframes on the stop_id and hour of the day, to add delay information to each edge
combined_df = edges.merge(delays,left_on=['simple_dst_id','hour'],right_on=['bpuic','hour'],how="left")\
                    .drop_duplicates(subset=["src", "dst", "src_departure", "dst_arrival", "trip_id"])\
                    .drop(columns=["bpuic", "simple_dst_id"])

## Infer delay information for missing values

Some of the edges might have not been matched with a delay. This is because we do not have delay information for all our nodes. To address that problem, we fill these missing delays with the average delay of lines from the same mean of transport. 

In [ ]:
# Map transport types to more general names
to_generic_transport = {
    "S-Bahn": "Train",
    "Bus": "Bus",
    "Tram": "Tram",
    "Standseilbahn": "Other", 
    "Schiff": "Other",
    "Luftseilbahn": "Other",
    "TGV": "Train",
    "Taxi": "Other",
    "Eurocity": "Train",
    "InterRegio": "Train",
    "ICE": "Train",
    "Intercity": "Train",
    "RegioExpress": "Train",
    "InterRegio": "Train"
}

In [ ]:
# Apply the more general transport type 
combined_df["type"] = combined_df.route_desc.apply(lambda x : to_generic_transport[x])

In [ ]:
# Compute average delay per hour and per general transport type
avg_delay_hour = combined_df.loc[:,("hour", "delay", "type")].groupby(["hour", "type"]).mean()
delay_dict = avg_delay_hour.to_dict()["delay"]

# Fill missing delay values with average of corresponding transport type
combined_df.delay = combined_df.apply(lambda x : delay_dict[(x.hour, x.type)] if pd.isna(x.delay) else x.delay, axis=1)

In [ ]:
# Drop the general type: we don't need it anymore
new_edges = combined_df.drop(columns="type")

In [ ]:
# Save the updated edges on hdfs
spark.createDataFrame(new_edges).write.mode("overwrite").option("compression","gzip").option("header", "True").csv("{}/delays.csv".format(folder_name))

In [ ]:
df_stop_line_delay.unpersist()